# Worksheet 08

Name:  Qicheng Cao
UID: U35901255

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [49]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[4.312784431995499, 4.819053234042214, 5.351588776617883, 5.542144492630738, 5.113798050000081, 4.568588572828494, 3.0742615992971447, 6.492499757848052, 5.695800261346071, 4.367602383166383]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [50]:
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)

[7.951054170476731, 7.032938636539395, 9.053770558019842, 6.471988329837584, 7.502991596004283, 7.137410969032196, 8.495094444215672, 9.639971592461217, 7.674638271018117, 8.80149864699442]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [51]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[8.80149864699442, 4.367602383166383, 7.674638271018117, 5.695800261346071, 6.492499757848052, 3.0742615992971447, 4.568588572828494, 5.113798050000081, 9.639971592461217, 8.495094444215672]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

Mean

Variance

$P(S_j)$

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [52]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[8.80149864699442, 7.674638271018117, 9.639971592461217, 8.495094444215672]
[4.367602383166383, 5.695800261346071, 6.492499757848052, 3.0742615992971447, 4.568588572828494, 5.113798050000081]
P(S_1) = 0.4,  P(S_2) = 0.6
mean_1 = 8.652800738672356,  mean_2 = 4.885425104081038
var_1 = 0.49457261273706843,  var_2 = 1.1567316732688524


/Users/luling/anaconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Mean is close to the true value, but variance is not.

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [53]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0] * prob_s[0] / prob_x )
    prob_s1_x.append( pdf_i[1] * prob_s[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  8.80149864699442
probability of observing that point if it came from cluster 0 =  0.7709935473498541
probability of observing that point if it came from cluster 1 =  0.001119111071778656
point =  4.367602383166383
probability of observing that point if it came from cluster 0 =  4.025888173409656e-17
probability of observing that point if it came from cluster 1 =  0.31200475167743597
point =  7.674638271018117
probability of observing that point if it came from cluster 0 =  0.11409650253660639
probability of observing that point if it came from cluster 1 =  0.01884152868106249
point =  5.695800261346071
probability of observing that point if it came from cluster 0 =  1.3940002366990203e-08
probability of observing that point if it came from cluster 1 =  0.26983667484379553
point =  6.492499757848052
probability of observing that point if it came from cluster 0 =  5.802374712572839e-05
probability of observing that point if it came from cluster 1 =  0.13138187489176503
point =  

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [54]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]
var = [sum([x[0] * (x[1] - mean[0])**2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * (x[1] - mean[1])**2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.4,  P(S_2) = 0.6
mean_1 = 8.70369218411947,  mean_2 = 4.979708252544034
var_1 = 0.46889740623796905,  var_2 = 1.3779308363474954


They are not very different, but they are closer to the true values.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [55]:
prob_c0_x = []  # P(C_0 | X_i)
prob_c1_x = []  # P(C_1 | X_i)
prob_x = []  # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x)
    prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x)

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  8.80149864699442
probability of observing that point if it came from cluster 0 =  0.8325002424180286
probability of observing that point if it came from cluster 1 =  0.006183490565404275
point =  4.367602383166383
probability of observing that point if it came from cluster 0 =  2.2937540872531964e-19
probability of observing that point if it came from cluster 1 =  0.26232064713975023
point =  7.674638271018117
probability of observing that point if it came from cluster 0 =  0.07655418969097977
probability of observing that point if it came from cluster 1 =  0.042763948659960936
point =  5.695800261346071
probability of observing that point if it came from cluster 0 =  9.868355342219514e-10
probability of observing that point if it came from cluster 1 =  0.25295124269219765
point =  6.492499757848052
probability of observing that point if it came from cluster 0 =  1.2615169902273198e-05
probability of observing that point if it came from cluster 1 =  0.15847113185037587
point =

It is more likely to be in cluster 1 than cluster 2.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [56]:
c1 = [x[0] for x in filter(lambda x: x[1] > x[2], zip(data, prob_c0_x, prob_c1_x))]
c2 = [x[0] for x in filter(lambda x: x[1] < x[2], zip(data, prob_c0_x, prob_c1_x))]
print(f"points in cluster 0 : {c1}\npoints in cluster 1 : {c2}")


points in cluster 0 : [8.80149864699442, 7.674638271018117, 9.639971592461217, 8.495094444215672]
points in cluster 1 : [4.367602383166383, 5.695800261346071, 6.492499757848052, 3.0742615992971447, 4.568588572828494, 5.113798050000081]
